In [141]:
import pandas as pd

report = pd.read_csv("../database/annualReports/ATT_Y_V.csv")


In [145]:
report["Unnamed: 0"].to_list()[1:]

['Przychody ze sprzedaży',
 'Techniczny koszt wytworzenia produkcji sprzedanej',
 'Koszty sprzedaży',
 'Koszty ogólnego zarządu',
 'Zysk ze sprzedaży',
 'Pozostałe przychody operacyjne',
 'Pozostałe koszty operacyjne',
 'Zysk operacyjny (EBIT)',
 'Przychody finansowe',
 'Koszty finansowe',
 'Pozostałe przychody (koszty)',
 'Zysk z działalności gospodarczej',
 'Wynik zdarzeń nadzwyczajnych',
 'Zysk przed opodatkowaniem',
 'Zysk (strata) netto z działalności zaniechanej',
 'Zysk netto',
 'Zysk netto akcjonariuszy jednostki dominującej',
 'Aktywa trwałe',
 'Wartości niematerialne i prawne',
 'Rzeczowe składniki majątku trwałego',
 'Należności długoterminowe',
 'Inwestycje długoterminowe',
 'Pozostałe aktywa trwałe',
 'Aktywa obrotowe',
 'Zapasy',
 'Należności krótkoterminowe',
 'Inwestycje krótkoterminowe',
 'Środki pieniężne i inne aktywa pieniężne',
 'Pozostałe aktywa obrotowe',
 'Aktywa trwałe przeznaczone do sprzedaży',
 'Aktywa razem',
 'Kapitał własny akcjonariuszy jednostki dominuj

### Read date database and mapper

In [5]:
from scripts import DateFinanceReportDB
import pickle

dateDB = DateFinanceReportDB()
dateDB.loadDatabase("../database/reportsDate/dateDB")
date = dateDB.getDict()

with open("../database/reportsDate/tickerMapper", 'rb') as f:
    ticker2dateName = pickle.load(f)


### Preparation data

In [12]:
report.iloc[0]

Unnamed: 0    Period
0             2006.0
1             2007.0
2             2008.0
3             2009.0
4             2010.0
5             2011.0
6             2012.0
7             2013.0
8             2014.0
9             2015.0
10            2016.0
11            2017.0
12            2018.0
13            2019.0
14            2020.0
15            2021.0
16            2022.0
17            2023.0
Name: 0, dtype: object

In [8]:
date[ticker2dateName["ATT"]]["Roczny"]

[[19, 6, 2008],
 [30, 4, 2009],
 [30, 4, 2010],
 [21, 3, 2011],
 [20, 3, 2012],
 [21, 3, 2013],
 [21, 3, 2014],
 [16, 3, 2015],
 [11, 3, 2016],
 [28, 4, 2017],
 [19, 4, 2018],
 [25, 4, 2019],
 [8, 4, 2020],
 [15, 4, 2021],
 [27, 4, 2022],
 [30, 3, 2023],
 [29, 4, 2024]]

In [17]:
import numpy as np
pd.to_datetime("19/6/2008")

/tmp/ipykernel_171253/3634496330.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pd.to_datetime("19/6/2008")


Timestamp('2008-06-19 00:00:00')

In [25]:
def castDate(date_list : list) -> np.datetime64:
    return np.datetime64(f"{date_list[-1]:04}-{date_list[-2]:02}-{date_list[-3]:02}")

In [138]:
reports_path = "../database/annualReports/"
end_subset = "_Y_V.csv"

ticker = "ATT"

# Prepare list of ticker to parse
wigs_list = pd.read_csv("../database/gpwCompaniesLists/WIGs.csv")
ticker_list = wigs_list.to_numpy().flatten()
ticker_list = ticker_list[~pd.isnull(ticker_list)]

merged_data = []
for ticker in ticker_list:
    # Read report
    report = pd.read_csv(reports_path+ticker+end_subset).to_numpy().transpose()[1:]
    report_years = report[:,0].astype(int)

    publication_date = np.array(date[ticker2dateName[ticker]]["Roczny"])
    publication_years = publication_date[:,2]

    for i, report_year in enumerate(report_years):
        for j, publication_year in enumerate(publication_years):
            if report_year == publication_year:
                row = report[i,:]
                row[0] = castDate(publication_date[j])
                row = np.insert(row, 0, ticker)
                merged_data.append(row)
                break



In [157]:
columns_name = np.append(["Ticker", "Data"], report["Unnamed: 0"].to_list()[1:])

In [162]:
merged_data_df = pd.DataFrame(merged_data)
merged_data_df.columns = columns_name
merged_data_df

,Ticker,Data,Przychody ze sprzedaży,Techniczny koszt wytworzenia produkcji sprzedanej,Koszty sprzedaży,Koszty ogólnego zarządu,Zysk ze sprzedaży,Pozostałe przychody operacyjne,Pozostałe koszty operacyjne,Zysk operacyjny (EBIT),...,Rozliczenia międzyokresowe,Pasywa razem,Przepływy pieniężne z działalności operacyjnej,Amortyzacja,Przepływy pieniężne z działalności inwestycyjnej,CAPEX (niematerialne i rzeczowe),Przepływy pieniężne z działalności finansowej,Emisja akcji,Dywidenda,Przepływy pieniężne razem
0,ATT,2008-06-19,1352156.0,1108307.0,61054.0,97418.0,85377.0,25691.0,21933.0,89135.0,...,0.0,1604246.0,62174.0,72887.0,-380990.0,145812.0,309736.0,285571.0,1000.0,-9080.0
1,ATT,2009-04-30,1214638.0,1051564.0,64713.0,99720.0,-1359.0,15791.0,28785.0,-14353.0,...,0.0,1581318.0,-43288.0,80844.0,-222648.0,75813.0,21792.0,0.0,42459.0,-244144.0
2,ATT,2010-04-30,1901784.0,1572293.0,82123.0,123834.0,123534.0,391040.0,92368.0,422206.0,...,0.0,3246701.0,115161.0,92328.0,-63996.0,94979.0,-16393.0,0.0,0.0,34772.0
3,ATT,2011-03-21,5338029.0,4151389.0,151617.0,312087.0,722936.0,34000.0,175609.0,581327.0,...,0.0,4946229.0,-167125.0,188864.0,-338520.0,344034.0,462189.0,594278.0,0.0,-43456.0
4,ATT,2012-03-20,7098735.0,6107680.0,181916.0,390733.0,418406.0,49024.0,95698.0,371732.0,...,0.0,5357041.0,543780.0,239592.0,-596148.0,397294.0,45970.0,0.0,0.0,-6398.0
5,ATT,2013-03-21,9821023.0,8436709.0,447750.0,706391.0,230173.0,601010.0,128558.0,702625.0,...,0.0,9961530.0,990497.0,550166.0,-688538.0,911834.0,169451.0,0.0,170813.0,471410.0
6,ATT,2014-03-21,9898476.0,8231565.0,581145.0,739977.0,345789.0,58421.0,101980.0,302230.0,...,0.0,9948332.0,819781.0,520175.0,-648066.0,676398.0,-328041.0,0.0,41768.0,-156326.0
7,ATT,2015-03-16,10024405.0,7760211.0,633314.0,736546.0,894334.0,47664.0,111987.0,830011.0,...,0.0,10777169.0,1630226.0,484810.0,-1503373.0,1061762.0,68518.0,0.0,25544.0,195371.0
8,ATT,2016-03-11,8955690.0,6997921.0,669315.0,729629.0,558825.0,47379.0,120142.0,486062.0,...,0.0,11051920.0,1115806.0,519721.0,-1303190.0,1241240.0,74395.0,0.0,96522.0,-112989.0
9,ATT,2017-04-28,9617495.0,7457734.0,673555.0,757767.0,728439.0,50200.0,181425.0,597214.0,...,0.0,11738044.0,1114225.0,589672.0,-709222.0,1069140.0,54490.0,0.0,100313.0,459493.0


In [165]:
merged_data_df.loc[merged_data_df['Ticker'] == "ATT"]

,Ticker,Data,Przychody ze sprzedaży,Techniczny koszt wytworzenia produkcji sprzedanej,Koszty sprzedaży,Koszty ogólnego zarządu,Zysk ze sprzedaży,Pozostałe przychody operacyjne,Pozostałe koszty operacyjne,Zysk operacyjny (EBIT),...,Rozliczenia międzyokresowe,Pasywa razem,Przepływy pieniężne z działalności operacyjnej,Amortyzacja,Przepływy pieniężne z działalności inwestycyjnej,CAPEX (niematerialne i rzeczowe),Przepływy pieniężne z działalności finansowej,Emisja akcji,Dywidenda,Przepływy pieniężne razem
0,ATT,2008-06-19,1352156.0,1108307.0,61054.0,97418.0,85377.0,25691.0,21933.0,89135.0,...,0.0,1604246.0,62174.0,72887.0,-380990.0,145812.0,309736.0,285571.0,1000.0,-9080.0
1,ATT,2009-04-30,1214638.0,1051564.0,64713.0,99720.0,-1359.0,15791.0,28785.0,-14353.0,...,0.0,1581318.0,-43288.0,80844.0,-222648.0,75813.0,21792.0,0.0,42459.0,-244144.0
2,ATT,2010-04-30,1901784.0,1572293.0,82123.0,123834.0,123534.0,391040.0,92368.0,422206.0,...,0.0,3246701.0,115161.0,92328.0,-63996.0,94979.0,-16393.0,0.0,0.0,34772.0
3,ATT,2011-03-21,5338029.0,4151389.0,151617.0,312087.0,722936.0,34000.0,175609.0,581327.0,...,0.0,4946229.0,-167125.0,188864.0,-338520.0,344034.0,462189.0,594278.0,0.0,-43456.0
4,ATT,2012-03-20,7098735.0,6107680.0,181916.0,390733.0,418406.0,49024.0,95698.0,371732.0,...,0.0,5357041.0,543780.0,239592.0,-596148.0,397294.0,45970.0,0.0,0.0,-6398.0
5,ATT,2013-03-21,9821023.0,8436709.0,447750.0,706391.0,230173.0,601010.0,128558.0,702625.0,...,0.0,9961530.0,990497.0,550166.0,-688538.0,911834.0,169451.0,0.0,170813.0,471410.0
6,ATT,2014-03-21,9898476.0,8231565.0,581145.0,739977.0,345789.0,58421.0,101980.0,302230.0,...,0.0,9948332.0,819781.0,520175.0,-648066.0,676398.0,-328041.0,0.0,41768.0,-156326.0
7,ATT,2015-03-16,10024405.0,7760211.0,633314.0,736546.0,894334.0,47664.0,111987.0,830011.0,...,0.0,10777169.0,1630226.0,484810.0,-1503373.0,1061762.0,68518.0,0.0,25544.0,195371.0
8,ATT,2016-03-11,8955690.0,6997921.0,669315.0,729629.0,558825.0,47379.0,120142.0,486062.0,...,0.0,11051920.0,1115806.0,519721.0,-1303190.0,1241240.0,74395.0,0.0,96522.0,-112989.0
9,ATT,2017-04-28,9617495.0,7457734.0,673555.0,757767.0,728439.0,50200.0,181425.0,597214.0,...,0.0,11738044.0,1114225.0,589672.0,-709222.0,1069140.0,54490.0,0.0,100313.0,459493.0
